# Imports

In [1]:
from fastai.imports import *
from bs4 import BeautifulSoup
from tqdm import tqdm

# Driver Functions

In [2]:
def list_of_brands():
    
    brands_ = []
    map_brands = {}
    url_2 = "https://www.next.co.uk/clearance/search?w=*&af=gender:men"
    page = requests.get(url_2)
    soup = BeautifulSoup(page.text, 'html.parser').prettify()

    a = soup.find("saleclearance.pageState.filters = ")
    b = soup.find("saleclearance.pageState.filters_TotalResults")
    aa = soup[a+len("saleclearance.pageState.filters = "):b].strip()[:-1]
    list_of_filters = json.loads(aa)
    for obj_brand in list_of_filters[3]['FilterOptions']:
        map_brands[obj_brand['Value']] = obj_brand['Name'].lower()
        brands_.append(obj_brand['Value'])

    return map_brands, brands_

In [3]:
def find_total_results(soup):
    
    text_for_total = "saleclearance.pageState.filters_TotalResults = "
    pos = soup.find(text_for_total) + len(text_for_total)
    res = ""
    while(soup[pos]!= ';'):
        res += soup[pos]
        pos +=1
    
    return int(res)

In [4]:
def source_to_data(soup):
    
    start = "saleclearance.pageState.results = "
    end = "saleclearance.pageState.isLandingPage = false"
    start_pos = soup.find(start) + len(start)
    end_pos = soup.find(end)
    return str(soup[start_pos:end_pos]).strip()[:-1]

In [5]:
def parse_products_of_brand(url, brand):
    
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser').prettify()
    total_results = find_total_results(soup)
    
    no_pages=total_results//24 + (1 if total_results%24 > 0 else 0)
    
    pages = [i+1 for i in range(no_pages)]
    str_list = [(i-1)*24 for i in pages]
    
    for str_no, page_no in zip(str_list, pages):
        url_page = url + f"%20&srt={str_no}#searchpos_{page_no}"
        per_page = requests.get(url_page)
        soup_per_page = BeautifulSoup(per_page.text, 'html.parser').prettify()
        
        try:
            list_of_products = json.loads(source_to_data(soup_per_page))
        except e:
            print(e)
        
        for product in list_of_products:
            original_price = product['ItemOptions'][0]['OriginalPrice']
            price = product['ItemOptions'][0]['Price']
            image_url = product['SearchImage'] + product['Image']
            data.append([product['Name'], brand, float(original_price), float(price), image_url])

In [6]:
def add_to_df():
    
    global df
    cols = ['Name', 'Brand', 'Original Price', 'Discounted Price', 'Image Url']
    df = pd.DataFrame(data, columns=cols)
    df.to_csv('output.csv', index=False)

In [7]:
def main():
    
    global data
    data = []
    url_main = "https://www.next.co.uk/clearance/search?w=*&af=gender:men"
    map_brands, brands = list_of_brands()
    
    for brand in tqdm(brands):
        brand_link = f"%20{brand}%20gender:men"
        link = url_main + brand_link
        parse_products_of_brand(link, map_brands[brand])
        
    add_to_df()

In [8]:
if __name__ == '__main__':
    main()

100%|██████████| 84/84 [03:52<00:00,  2.76s/it]


# EOF